This takes letters I've written to my daughters over the years and creates a chatbot they can ask questions to and receive answers based on those letters.
The larger goal is to provide a chatbot that can answer my kid's questions, even when I'm not around (the notebook is named "appa" because this is the Korean word for dad).

It uses OpenAI's gpt-3.5-turbo-0125 model, Chroma to store embedded documents, and LangChain to tie things together.

In [16]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser


LETTERS_PATH = "txt"
CHROMA_PATH = "/tmp/chroma_data"

load_dotenv()

chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [17]:
loader = DirectoryLoader(path=LETTERS_PATH)
vector_db = Chroma.from_documents(
    loader.load(), OpenAIEmbeddings(), persist_directory=CHROMA_PATH
)

In [31]:
question = "How can I become more confident?"

In [32]:
template = """Your job is to provide wise answers to questions. Use
the following context to answer questions. Be as detailed as possible, but
don't make up any information that's not from the context. If you don't know
an answer, say "I'm based off of your Dad's letters to you and unfortunately, 
I don't have any letters with context that match your question"
{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["context"], template=template)
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [review_system_prompt, review_human_prompt]
prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
chain = (
    {"context": vector_db.as_retriever(k=3), "question": RunnablePassthrough()}
    | prompt_template
    | chat_model
    | StrOutputParser()
)

In [34]:
response = chain.invoke(question)

In [35]:
print(response)

To become more confident, you can start by following these steps:

1. **Think critically**: Reflect on your beliefs and values. Understand what you stand for and why. This self-awareness will boost your confidence.

2. **Trust your instincts**: Listen to your inner voice. Your instincts are often a result of your experiences and can guide you in making confident decisions.

3. **Embrace failure**: Failure is a part of growth. Instead of being afraid of it, see it as a learning opportunity. Each failure brings you closer to success.

4. **Ignore the opinions of others**: Don't let the fear of judgment hold you back. Focus on being true to yourself rather than seeking validation from others.

5. **Experiment boldly**: Step out of your comfort zone and try new things. The more you challenge yourself, the more confident you will become in your abilities.

6. **Surround yourself with supportive people**: Build a circle of friends and colleagues who appreciate and encourage your boldness. Po

Interestingly, the bold sections (those within **) are verbatim from my letters.